## Lab 3: **(n<sup>2</sup>-1)-puzzle**

In [451]:
import random

## **Problem representation**
The board numbers disposition is represented as a list of integers and is interpreted as a matrix with row major technique<br>
For example, in the case of a 3x3 matrix the related vector has length 9.<br>
The actions are made by swapping the zero with one of the possible adjancent numbers.

In [452]:
PUZZLE_DIM = 2
correct_solution = list()

# function applying an action to the current board disposition
def do_action(disp, action):
    new_disp = disp.copy()
    new_disp[action[0]] = disp[action[1]]
    new_disp[action[1]] = 0

    return new_disp


# function retrieving all the possible actions related to the current board disposition
def available_actions(disp):
    actions = list()
    zero_pos = -1

    for i in range(PUZZLE_DIM**2):
        if disp[i] == 0:
            zero_pos = i
            break
    
    # zero shifted to the right
    if (zero_pos + 1) % PUZZLE_DIM != 0:
        actions.append((zero_pos, zero_pos + 1))
    # zero shifted to the left
    if zero_pos % PUZZLE_DIM != 0:
        actions.append((zero_pos, zero_pos - 1))
    # zero shifted to the top
    if zero_pos >= PUZZLE_DIM:
        actions.append((zero_pos, zero_pos - PUZZLE_DIM))
    # zero shifted to the bottom
    if zero_pos < (PUZZLE_DIM**2 - PUZZLE_DIM):
        actions.append((zero_pos, zero_pos + PUZZLE_DIM))
    
    return actions


# function verifying whether the current board disposition is the solution to the problem
def is_solution(disp):
    return disp == correct_solution

Definition of the correct (final) solution and generation of the starting disposition of the board, which is obtained by executing 10.000 random actions on the correct solution.

In [453]:
# definition of the final (correct) disposition
disp = [i for i in range(1, PUZZLE_DIM**2)]
disp.append(0)
correct_solution = disp.copy()

# random iniztialization of the board disposition
RANDOMIZE_STEPS = 10_000
for i in range(10_000):
    actions = available_actions(disp)
    rand_action = actions[random.randint(0, len(actions) - 1)]
    disp = do_action(disp, rand_action)

print("Initial state:")
print(disp)

Initial state:
[0, 3, 2, 1]


## Approach 1: **Breadth-first search**
This approach implements a breadth-first search that uses a tree of visits instead of the fronteer queue. Each time a new node is discovered it is added to the tree of the visits, which is stored as a list and whose nodes are instances of the Node class and have the following attributes:
- `disp`: the disposition of the numbers on the board (also represented as a list but interpreted as a matrix with row major)
- `parent`: the index of the tree in which we can find the parent of the node (the default value for the root element is `-1`, in order to recognize it)

The parent index is needed for each node since the tree is not necessarily binary or quasi-complete, so we cannot use a heap structure. The tree of visits is then used to "reconstruct" the path by starting from the final solution and "climbing" the tree up to the root element.

In [454]:
# class representing the single node of the tree of the visits
class Node:
    def __init__(self, disp, parent):
        self.disp = disp
        self.parent = parent

In [455]:
# initialization of the problem tree
TREE = list()
# the tree initially contains only the starting state (root node)
TREE.append(Node(disp.copy(), -1))
# the tree visit starts from the root node (at index 0)
visit_index = 0
state = TREE[visit_index]

found = is_solution(disp)
# the tree is visited iteratively with a breadth-first search
while not found:
    # possible actions that can be performed starting from the current state
    actions = available_actions(state.disp)
    # initialization of the parent of the parent (if present)
    ancestor_state = None
    if state.parent != -1:
        ancestor_state = TREE[state.parent]
    
    # the states resulting from the possible actions are added to the tree
    for i in range(len(actions)):
        new_state = Node(do_action(state.disp, actions[i]), visit_index)
        # the new state is added as a node of the tree only if it does not bring the board to a
        # previous disposition
        if ancestor_state == None or new_state.disp != ancestor_state.disp:
            TREE.append(new_state)
        # if the newly added node has a disposition which corresponds to the solution then the 
        # tree visit stops
        if is_solution(new_state.disp):
            found = True
            break
    
    if not found:
        # the next node to visit is the one following the last one visited in the tree
        visit_index += 1
        state = TREE[visit_index]


# the solution path is reconstructed starting from the found solution and "climbing" the tree list going from
# children to parent until the root element (initial disposition) is reached
path = []
path_index = len(TREE) - 1
while path_index != -1:
    path.insert(0, TREE[path_index].disp)
    path_index = TREE[path_index].parent

# output print
print("Number of steps: " + str(len(path) - 1))
print("Path:")
for step in path:
    print(step)

Number of steps: 6
Path:
[0, 3, 2, 1]
[3, 0, 2, 1]
[3, 1, 2, 0]
[3, 1, 0, 2]
[0, 1, 3, 2]
[1, 0, 3, 2]
[1, 2, 3, 0]


## Approach 2: **Depth-first search with backtrack**
This approach implements a depth-first search that uses the tree of visits as defined in the previous algorithm. This time each Node instance has the following attributes:
- `disp`: the disposition of the numbers on the board (also represented as a list but interpreted as a matrix with row major)
- `parent`: the index of the tree in which we can find the parent of the node (the default value for the root element is `-1`, in order to recognize it)
- `visited`: boolean value used to know whether the node has been visited (used for backtrack)

The backtrack is implemented in order to allow the algorithm to "escape" from those situations where a branch has no more possible actions so it keeps cycling on the same node in an infinite loop. The backtrack allows to "climb" the tree until the first (from the bottom) node that has been discovered but not visited yet, in order to continue the visit from there.

In [456]:
# class representing the single node of the tree of the visits
class Node:
    def __init__(self, disp, parent):
        self.disp = disp
        self.parent = parent
        self.visited = False

In [457]:
# initialization of the problem tree
TREE = list()
# the tree initially contains only the starting state (root node)
TREE.append(Node(disp.copy(), -1))
# the tree visit starts from the root node (at index 0)
state = TREE[0]
# index of the current parent
current_parent = 0

found = is_solution(disp)
# the tree is visited iteratively with a breadth-first search
while not found:
    # we mark the node as visited
    state.visited = True
    # possible actions that can be performed starting from the current state
    actions = available_actions(state.disp)
    # initialization of the parent of the parent (if present)
    ancestor_state = None
    if state.parent != -1:
        ancestor_state = TREE[state.parent]
    
    # the states resulting from the possible actions are added to the tree
    for i in range(len(actions)):
        new_state = Node(do_action(state.disp, actions[i]), current_parent)
        # the new state is added as a node of the tree only if it does not bring the board to a
        # previous disposition
        if new_state.disp not in map(lambda el: el.disp, TREE):
            TREE.append(new_state)
        # if the newly added node has a disposition which corresponds to the solution then the 
        # tree visit stops
        if is_solution(new_state.disp):
            found = True
            break
    
    if not found:
        # the next node to visit is the last one inserted in the tree
        state = TREE[len(TREE) - 1]
        cnt = 0
        while state.parent == -1 or state.visited == True:
            cnt += 1
            state = TREE[len(TREE) - 1 - cnt]
        current_parent = len(TREE) - 1 - cnt


# the solution path is reconstructed starting from the found solution and "climbing" the tree list going from
# children to parent until the root element (initial disposition) is reached
path = []
path_index = len(TREE) - 1
while path_index != -1:
    path.insert(0, TREE[path_index].disp)
    path_index = TREE[path_index].parent

# output print
print("Number of steps: " + str(len(path) - 1))
print("Path:")
for step in path:
    print(step)

Number of steps: 6
Path:
[0, 3, 2, 1]
[2, 3, 0, 1]
[2, 3, 1, 0]
[2, 0, 1, 3]
[0, 2, 1, 3]
[1, 2, 0, 3]
[1, 2, 3, 0]


## Approach 3: **A-star algorithm**
This approach is based on a A-star algorithm, where the cost function *f(n) = g(n) + h(n)*, for each node n, is based on:
- ***g(n)***: total number of moves needed to reach that node
- ***h(n)*** (*euristhic function*): Manhattan distance of the disposition of the node

The cost *f(n)* must be as little as possible and the fronteer is sorted with a descending order of cost, so that the node extraction can be made from the tail of the list and not from the head (it's a faster operation).<br>
The visited nodes are marked in a dictionary whose keys are the strings of the node numbers disposition. This is made in order to make the searching of an already visited node faster exploiting the random access and avoiding a O(n) list search.

In [458]:
# definition of the euristhic function, which is based on the so-called Manhattan distance, which measures the overall
# difference between the position of each number in the current state and the one in the correct (final) state
def manhattan_distance(disp):
    sum = 0
    for val in range(1, PUZZLE_DIM**2):
        x_sol = (val - 1) // PUZZLE_DIM
        y_sol = (val - 1) % PUZZLE_DIM

        i = -1
        for j in range(PUZZLE_DIM**2):
            if disp[j] == val:
                i = j
                break
        x = i // PUZZLE_DIM
        y = i % PUZZLE_DIM

        sum += (abs(x - x_sol) + abs(y - y_sol))
    
    return sum

In [459]:
# class representing the single node of the search
class Node:
    def __init__(self, disp, moves):
        self.disp = disp
        self.moves = moves
        self.cost = moves + manhattan_distance(disp)

In [460]:
# list containing the states explored but not yet visited
fronteer = list([Node(disp, 0)])
# dictionary containing the states explored and visited
visited = dict()
# list containing the path of the solution
path = list()

state = fronteer.pop()
path.append(state.disp)

while state.disp != correct_solution:
    visited["".join(map(str, state.disp))] = state.cost
    # possible actions that can be performed starting from the current state
    actions = available_actions(state.disp)

    for i in range(len(actions)):
        new_state = Node(do_action(state.disp, actions[i]), state.moves + 1)
        # the new state is discarded if its disposition is already in the visited dictionary and its cost is greater
        key = "".join(map(str, new_state.disp))
        if (key not in visited) or (key in visited and new_state.cost > visited[key]):
            # the new state is added to the fronteer
            fronteer.append(new_state)

    # the fronteer is sorted according to the cost of the nodes
    fronteer.sort(key=lambda el: el.cost, reverse=True)
    state = fronteer.pop()
    # the current state is added to the solution path
    path.append(state.disp)


# output print
print("Number of steps: " + str(len(path) - 1))
print("Path:")
for step in path:
    print(step)

Number of steps: 6
Path:
[0, 3, 2, 1]
[2, 3, 0, 1]
[2, 3, 1, 0]
[2, 0, 1, 3]
[0, 2, 1, 3]
[1, 2, 0, 3]
[1, 2, 3, 0]
